# Welcome to the Workshop: Software Tools from Atomistics to Phase Diagrams 
https://pyiron.org - https://www.mpie.de - https://workshop.materialsgenomefoundation.org
* Richard Otis (Materials Genome Foundation)
* **Jan Janssen**, Tilmann Hickel, Jörg Neugebauer (Max-Planck-Institut für Eisenforschung)
* Brandon Bocklund and Zi-Kui Liu (Pennsylvania State University)

# Day 1 - atomistic simulation 
12:30-13:00 Interactive: **Density functional theory with pyiron**

The scope of this first exercise is to become familar with:
* Density functional theory calculation, 
* Calculate the equilibrium lattice constant 
* Compare the results to interatomic potentials.

## Reminder
In the first session we learned how to create a pyiron project object and then use this pyiron project object to create atomistic structure objects. 

In [ ]:
# Import the Project object
from pyiron import ____

# Create a Project object instance for a project named atomistics
pr = ____("simulations")

# Create a cubic aluminum fcc structure
al_fcc = pr.create_ase_bulk(____, _____=True)

# Confirm the final structure has 4 atoms by calculating the length of the structure object
____(al_fcc_repeated) == 4

# Density Functional Theory 
To provide a brief introduction to density functional theory (DFT) we calculate the equilibrium lattice constant for a cubic fcc aluminium structure. Besides the pseudo potential which defines the electron electron interaction, the DFT precision is dominated by the convergence parameters, namely the plane wave energy cutoff and the kpoint mesh. Both can be set in pyiron using the corresponding properties `encut` and `kpoint_mesh`.  

In [ ]:
# Create a DFT job with the S/PHI/nX quantum engine named spx
job_dft_template = pr.create_job(
    job_type=pr.job_type.Sphinx,
    job_name=________
)

# Print the default DFT convergence parameters energy cutoff
print(
    job_dft_template.encut, 
    job_dft_template.kpoint_mesh
)

# Increase the energy cutoff to 400eV and increase the kpoint mesh to 5x5x5
job_dft_template.set_encut(_______)
job_dft_template.set_kpoints([__, __, __])

# Again we enable the run_mode non_modal to execute the calculation in the background
job_dft_template.server.run_mode.non_modal = ______

## Energy Volume Curve 
The discretisation of the plane waves on the fourier mesh and the kpoint mesh in the brillouin zone cause fluctuations in the energy surface. Therefore a minimization like we used it for the interatomic potentials in the previous section is insufficient. Instead we calculate the energy for various volumes around the equlibrium volume and then determine the equilibrium volume by interpolating the minimum between these volumes.

In [ ]:
# We import the numerical library numpy
import numpy as np

# We create 7 strains ranging from -5% (0.95) to +5% (1.05) 
for strain in np.linspace(_____, ______, _____):
    # We include the strain in the job name to identify the individual jobs 
    job_strain = job_dft_template.copy_to(
        new_job_name='spx_' + str(strain).replace('.', '_'), 
        new_database_entry=False
    )
    
    # Each job we assign a copy of the cubic fcc aluminium supercell
    job_strain.structure = _______.copy()
    
    # Afterwards we rescale the volume of the supercell with the selected strain
    job_strain.structure.set_cell(cell=al_fcc.cell * _______ ** (1/3), scale_atoms=True)
    
    # Finally we submit the calculation for execution 
    job_strain.run()

In [ ]:
# While the calculations are executed in the background we check the status of the calculation in the job_table
pr.job_table()

In [ ]:
# We wait until all calculations are finshed 
pr.wait_for_jobs()

## Reminder
We again use the pyiron table object to collect the simulation results and aggregate them in a pandas DataFrame. 

In [ ]:
# Create a pyiron table object
table = pr.create_table()

# Implement a filter function, which returns true for finished jobs and jobs with "spx_" in the job_name
def filter_jobs(job):
    return job.status == _______ and _____ in job.job_name

In [ ]:
# Many commonly used functions are already available using tab completion
# We select the get_volume function and the get_energy_tot function
table.add.________
table.add.get_energy_tot

# Assigne the filter function defined above
table.filter_function = _____

# Execute the pyiron table just like a pyiron job object
table._____

# Return a pandas DataFrame with the collected results 
df_res = table.get_dataframe()
df_res

## Visualise the energy volume curve 
We again use the matplotlib library to visualise the calculated energy volume curve and calculate the equilibrium volume by fitting a second order polynomial and calculate the roots of the derivative. 

In [ ]:
# Import the matplotlib library for plotting. As we are using jupyter notebooks 
# for combining code, documentation and visualisation, we also need the jupyter magic for matplotlib. 
# https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-matplotlib
%matplotlib inline 
import matplotlib.pyplot as plt

# Plot the volume and the total energy from the DataFrame with the collected results
plt.plot(_____.volume, _____.energy_tot)

# Fit a second order polynomial to the energy volume curve 
fit = np.polyfit(_____.volume, _____.energy_tot, ___)

vol_eq = np.polyder(fit).roots[0]
plt.axvline(vol_eq, linestyle="--", color="red")
plt.xlabel("Volume")
plt.ylabel("total Energy");

## pyiron Master Jobs 
While managing calculations with with for loops and aggregating calculation results in pandas DataFrame is already a very scaleable concept, we have implemented master jobs which can execute multiple calculations in series or in parallel to automate common simulation tasks. The calculation of the energy volume curve is one of those examples. 

In [ ]:
# Create a DFT job with the S/PHI/nX quantum engine named "spxjob"
job_master_template = pr.create_job(
    job_type=pr.job_type.Sphinx,
    job_name=______
)

# Assign the cubic fcc aluminium structure to the template job 
job_master_template.structure = _____

# Again we enable the run_mode non_modal to execute the calculation in the background
job_master_template.server.run_mode.non_modal = ______

# Use the job object to create the Murnaghan object, named "murn" 
murn = job_master_template.create_job(
    job_type=pr.job_type.Murnaghan, 
    job_name=_______
)

# Set the number of cores of the Murnaghan object to 2 so it can execute two DFT calculations at a time 
murn.server.cores = _____

# Execute the Murnaghan object 
murn.run()

In [ ]:
# While the calculations are executed in the background we check the status of the calculation in the job_table
pr.job_table()

In [ ]:
# We again wait until all calculations are finshed 
pr.wait_for_jobs()

In [ ]:
# Finally we can use the internal functionality of the master job to visualise the energy volume curve
murn.plot()

## Summary 
In this section you learned: 
* how to calculate equilibrium bulk material properties using density functional theory.
* how the finite plane wave energy cutoff and the finite kpoint mesh limit the precision of DFT calculation. 
* and how to use master jobs in pyiron to automate common tasks like calculating energy volume curves. 

Suggestions: 
* To learn more about convergence you can plot the convergence of an individual energy over the change in plane wave energy cutoff or kpoint mesh. 
* To visualise the discretisation of the energy volume curve you can calculate the energy volume curve at a small volume range of +/- 1% for a low kpoint mesh of 3x3x3 and an energy cutoff of 300eV with 21 points.
* To validate the interatomic potentials from the previous section we can calculate energy volume curves for those and compare the energy differences to the DFT results. 